In [1]:
import os
import torch
import scipy.io
import argparse
import pandas as pd
import numpy as np

In [50]:
def compute_mAP(index, good_index, junk_index):
    ap = 0
    cmc = torch.IntTensor(len(index)).zero_()
    if good_index.size == 0:
        cmc[0] = -1
        return ap, cmc

    # remove junk_index
    mask = np.in1d(index, junk_index, invert=True)
    index = index[mask]

    # find good_index index
    ngood = len(good_index)
    mask = np.in1d(index, good_index)
    rows_good = np.argwhere(mask == True)
    rows_good = rows_good.flatten()

    cmc[rows_good[0]:] = 1
    for i in range(ngood):
        d_recall = 1.0 / ngood
        precision = (i + 1) * 1.0 / (rows_good[i] + 1)
        if rows_good[i] != 0:
            old_precision = i * 1.0 / rows_good[i]
        else:
            old_precision = 1.0
        ap = ap + d_recall * (old_precision + precision) / 2

    return ap, cmc[:20]

In [1]:
def evaluate(score, ql, qc, gl, gc):
    # predict index sort from small to large
    index = np.argsort(score)[::-1]
    # good index
    query_index = np.argwhere(gl == ql)
    camera_index = np.argwhere(gc == qc)
    good_index = np.setdiff1d(query_index, camera_index, assume_unique=True)
    #print(good_index)
    junk_index1 = np.argwhere(gl == -1)
    junk_index2 = np.intersect1d(query_index, camera_index)
    junk_index = np.append(junk_index2, junk_index1)
    #print(junk_index)
    top_labels = []
    actual_labels = []
    for val in index[:10]:
        top_labels.append(gl[val])
    for val in good_index:
        actual_labels.append(gl[val])
    print(index[:10], top_labels)
    return compute_mAP(index, good_index, junk_index)


In [2]:
torch.cuda.set_device(0)
logPath = '../../data/Market/extracted'
labelPath = '../../data/Market/extracted'

logFile = {subset: scipy.io.loadmat(os.path.join(logPath, 'feature_val_%s.mat' % subset))
           for subset in ['query', 'gallery']}
logFile_uc = {subset: scipy.io.loadmat(os.path.join(logPath, 'feature_uc_val_%s.mat' % subset))
           for subset in ['query', 'gallery']}
labelDict = {subset: pd.read_csv(os.path.join(labelPath, subset + 'Info.txt'), header=None,
                                 delimiter='\t').set_index(0)[1].to_dict() for subset in ['query', 'gallery']}

names = {subset: logFile[subset]['names'] for subset in ['query', 'gallery']}
names_uc = {subset: logFile_uc[subset]['names'] for subset in ['query', 'gallery']}
labels = {subset: np.array([labelDict[subset][name] for name in names[subset]]) for subset in ['query', 'gallery']}
labels_uc = {subset: np.array([labelDict[subset][name] for name in names_uc[subset]]) for subset in ['query', 'gallery']}
features = {subset: torch.FloatTensor(logFile[subset]['features']).cuda() for subset in ['query', 'gallery']}
features_uc = {subset: torch.FloatTensor(logFile_uc[subset]['features']).cuda() for subset in ['query', 'gallery']}

NameError: name 'torch' is not defined

In [3]:
print(names['query'][:10], names_uc['query'][:10])

NameError: name 'names' is not defined

In [4]:
print(labels_uc)

NameError: name 'labels_uc' is not defined

In [5]:

'''uc_better = []
tc_better = []
uc_better_score = 0
tc_better_score = 0
CMC = torch.IntTensor(20).zero_()
ap = 0.0
CMC_better = torch.IntTensor(20).zero_()
ap_better = 0.0
CMC_uc = torch.IntTensor(20).zero_()
ap_uc = 0.0
#for i in range(len(labels['query'])):
for j in range(1):
    i = 6
    score = torch.mm(features['gallery'], features['query'][i].view(-1, 1))
    score = score.squeeze(1).cpu().numpy()
    ap_tmp, CMC_tmp = evaluate(score, labels['query'][i], [], labels['gallery'], [])
    score_uc = torch.mm(features_uc['gallery'], features_uc['query'][i].view(-1, 1))
    score_uc = score_uc.squeeze(1).cpu().numpy()
    ap_tmp_uc, CMC_tmp_uc = evaluate(score_uc, labels['query'][i], [], labels['gallery'], [])
    if ap_tmp_uc > ap_tmp:
        uc_better.append(i)
        uc_better_score += (ap_tmp_uc-ap_tmp)
        CMC_better += CMC_tmp_uc
        ap_better += ap_tmp_uc
    elif ap_tmp_uc < ap_tmp:
        tc_better.append(i)
        tc_better_score += (-ap_tmp_uc+ap_tmp)
        CMC_better += CMC_tmp
        ap_better += ap_tmp
    else:
        CMC_better += CMC_tmp
        ap_better += ap_tmp
    if CMC_tmp[0] == -1:
            continue
    CMC += CMC_tmp
    ap += ap_tmp
    CMC_uc += CMC_tmp_uc
    ap_uc += ap_tmp_uc
#print(CMC, CMC_better, CMC_uc, ap, ap_better, ap_uc)
'''

"uc_better = []\ntc_better = []\nuc_better_score = 0\ntc_better_score = 0\nCMC = torch.IntTensor(20).zero_()\nap = 0.0\nCMC_better = torch.IntTensor(20).zero_()\nap_better = 0.0\nCMC_uc = torch.IntTensor(20).zero_()\nap_uc = 0.0\n#for i in range(len(labels['query'])):\nfor j in range(1):\n    i = 6\n    score = torch.mm(features['gallery'], features['query'][i].view(-1, 1))\n    score = score.squeeze(1).cpu().numpy()\n    ap_tmp, CMC_tmp = evaluate(score, labels['query'][i], [], labels['gallery'], [])\n    score_uc = torch.mm(features_uc['gallery'], features_uc['query'][i].view(-1, 1))\n    score_uc = score_uc.squeeze(1).cpu().numpy()\n    ap_tmp_uc, CMC_tmp_uc = evaluate(score_uc, labels['query'][i], [], labels['gallery'], [])\n    if ap_tmp_uc > ap_tmp:\n        uc_better.append(i)\n        uc_better_score += (ap_tmp_uc-ap_tmp)\n        CMC_better += CMC_tmp_uc\n        ap_better += ap_tmp_uc\n    elif ap_tmp_uc < ap_tmp:\n        tc_better.append(i)\n        tc_better_score += (-ap_

In [6]:
print(uc_better, "   ", uc_better_score, "  ", len(uc_better))

NameError: name 'uc_better' is not defined

In [7]:
print(tc_better, "   ", tc_better_score, "  ", len(tc_better) )

NameError: name 'tc_better' is not defined

In [8]:
(features['gallery']+features_uc['gallery']).size()
(features['query']+features_uc['query']).size()

NameError: name 'features' is not defined

In [9]:
CMC = torch.IntTensor(20).zero_()
ap = 0.0
CMC_better = torch.IntTensor(20).zero_()
ap_better = 0.0
CMC_uc = torch.IntTensor(20).zero_()
ap_uc = 0.0
ap_map_values = {}
for i in range(len(labels['query'])):
#for j in range(1):
    #i = 19
    score = torch.mm(features_uc['gallery'], (features_uc['query'][i]).view(-1, 1))
    #score = torch.mm(features['gallery'], (features['query'][i]).view(-1, 1))
    score = score.squeeze(1).cpu().numpy()
    ap_tmp, CMC_tmp = evaluate(score, labels['query'][i], [], labels['gallery'], [])
    ap_map_values[labels['query'][i]] = ap_tmp
    if CMC_tmp[0] == -1:
            continue
    CMC += CMC_tmp
    ap += ap_tmp

NameError: name 'torch' is not defined

In [225]:
CMC = CMC.float()
CMC /= len(labels['query'])
ap /= len(labels['query'])

print('top1: %.4f, top5: %.4f, top10: %.4f, mAP: %.4f' % (CMC[0], CMC[4], CMC[9], ap))

top1: 0.8267, top5: 0.9133, top10: 0.9600, mAP: 0.6871


In [228]:
from collections import OrderedDict
from operator import itemgetter    
ap_map_values = (OrderedDict(sorted(ap_map_values.items(), key = itemgetter(1), reverse = False)))
print((ap_map_values))

OrderedDict([(84, 0.0038910309637632465), (105, 0.007044853812959166), (40, 0.023094051318796105), (139, 0.029788318586734076), (148, 0.0331538192439588), (36, 0.038212111573071486), (140, 0.04755481683231044), (118, 0.05608598708593367), (47, 0.06417353978628487), (1, 0.06677736006683374), (108, 0.06699148038662105), (54, 0.0675258742299251), (22, 0.09727077001013171), (138, 0.10073375262054507), (11, 0.10103874883286648), (103, 0.10246993404888141), (44, 0.14494301994301995), (107, 0.1549206349206349), (18, 0.21220238095238092), (142, 0.26068376068376065), (111, 0.2714285714285714), (48, 0.27400793650793653), (45, 0.29994468152362885), (144, 0.3194444444444444), (141, 0.33949017380351737), (12, 0.3471745123288008), (119, 0.3530394801454584), (7, 0.3574374779637937), (15, 0.3576331594752647), (34, 0.36398183484015567), (49, 0.37017003234548473), (6, 0.37222222222222223), (26, 0.3771576553313205), (37, 0.37777777777777777), (143, 0.38869047619047614), (9, 0.39383800563236043), (19, 0.3